# Azavea Climate Data

The following notebook summarizes the relevant portion of the Azavea climate API for AGC.

## Datasets

The datasets available on Azavea's service are
- [NASA NEX-GDDP](https://www.nccs.nasa.gov/services/climate-data-services)
- [UCSD LOCA](http://loca.ucsd.edu/)

The forecasted climate values are created assuming one of the following scenarios:
- [Representative Common Pathway 8.5 (RCP 8.5)](https://en.wikipedia.org/wiki/Representative_Concentration_Pathway#RCP_8.5)
    - Endpoint: "/api/climate-data/{location}/RCP85"
    - Spans 2006 - 2100
    - Azavea links to https://www.skepticalscience.com/rcp.php for more information
- Historical
    - Endpoint: "/api/climate-data/{location}/historical"
    - Spans 1950 - 2005

The climate data is also partitioned between these two endpoints. To see the full set of forecasted values from 1950 - 2100, they must be obtained by calling these two API endpoints separately.

## Climate Models

Each dataset provides a set of climate models using the above scenarios that are also provided by the dataset providers.

## Climate Indicators

The full list of available climate indicators can be found in the [documentation](https://docs.climate.azavea.com/indicators.html#)


In [28]:
import os
import json
from dotenv import load_dotenv
import requests
import time

from datetime import datetime
from dateutil.relativedelta import relativedelta

load_dotenv()

URL = "https://app.climate.azavea.com"

# This is the default list of selected Climate Models
# It has roughly half of the available models selected by default though I am unsure why it does not have them all selected
CLIMATE_MODELS = "ACCESS1-0,bcc-csm1-1,BNU-ESM,CanESM2,CCSM4,CESM1-BGC,CNRM-CM5,CSIRO-Mk3-6-0,GFDL-CM3,GFDL-ESM2G,GFDL-ESM2M,inmcm4,IPSL-CM5A-LR,IPSL-CM5A-MR,MIROC5,MIROC-ESM,MIROC-ESM-CHEM,MPI-ESM-LR,MPI-ESM-MR,MRI-CGCM3,NorESM1-M"

INDICATORS = [
    "average_high_temperature", 
    "average_low_temperature", 
    "cooling_degree_days", 
    "diurnal_temperature_range",
    "extreme_cold_events",
    "extreme_heat_events",
    "frost_days", 
    "heat_wave_duration_index",
    "heat_wave_incidents",
    "heating_degree_days",
    "max_high_temperature",
    "min_low_temperature",
    "percentile_high_temperature",
    "percentile_low_temperature",
    "dry_spells",
    "extreme_precipitation_events",
    "max_consecutive_dry_days",
    "percentile_precipitation",
    "total_precipitation",
]

INDICATORS_WITH_THRESHOLDS = [
    "max_temperature_threshold",
    "min_temperature_threshold",
    "precipitation_threshold",
]

HEADERS = {"Authorization": f"Token {os.getenv('CLIMATE_API_TOKEN')}"}


In [32]:
# This retrieves the "Average High Temperature" for the NASA NEX-GDDP Dataset using the climate models listed above
r = requests.get(f"{URL}/api/climate-data/1025/RCP85/indicator/dry_spells/?models={CLIMATE_MODELS}&time_aggregation=monthly&dataset=NEX-GDDP",
headers=HEADERS)
r.status_code

200

In [35]:
def trend(data: dict, delta: relativedelta):
    end_date = datetime.today()
    curr_date = end_date - delta
    new_data = {}
    while curr_date <= end_date:
        fmt_date = curr_date.strftime("%Y-%m")
        new_data[fmt_date] = data[fmt_date]
        curr_date += relativedelta(months=1)
    return new_data

climate_data = {}
for indicator in INDICATORS:
    # Note: No units are indicated so it will use whatever the endpoint's default unit is set to i.e Fahrenheit for "Average High Temperature"
    r = requests.get(f"{URL}/api/climate-data/1025/RCP85/indicator/{indicator}/?models={CLIMATE_MODELS}&time_aggregation=monthly&dataset=NEX-GDDP", headers=HEADERS)
    if r.status_code == 200:
        try:
            # Reduces the data to the interval [Current month 10 years ago, Current Month Present Year]
            data = r.json()["data"]
            data = trend(data, relativedelta(years=10))
            climate_data[indicator] = data
            with open(f"{indicator}.json", "w") as f:
                json.dump(data, f, indent=4)
        except KeyError:
            print(f"KeyError on \"{indicator}\". Available keys are: {r.json().keys()}")
    elif r.status_code == 429:
        sleep_seconds = int(r.headers["Retry-After"]) + 1
        print(f"Too Many Requests. Sleeping for {sleep_seconds} seconds")
        time.sleep(sleep_seconds)
    else:
        print(f"Retrieval for indicator \"{indicator}\" failed with HTTP status code: {r.status_code}")
    

with open("climate_data.json", "w") as f:
    json.dump(climate_data, f, indent=4)

Too Many Requests. Sleeping for 2
